In [2]:
import pandas as pd

df = pd.read_csv("data/tiktok_comments_final.csv")
df

,text,createTime
0,gibran:,1764933105
1,ini emg harusnya ganti rakyat sih. bgnian emg ...,1764928295
2,wapres aja gak pakek rompi anti peluru,1764922201
3,di indo doang presiden n wapres di hujat wkwk,1764925978
4,"di X parah bett anjir komenannya, merinding la...",1764922173
...,...,...
4188,Dikasih yg muda biar satset dihujat😭😭,1764934100
4189,Bahlil dan menteri kehutanan belike: Gibran sa...,1765114598
4190,"ini bagus tidak ada salahnya, semoga seterusny...",1765363698
4191,wapresnya cakep muda lagi 😂,1765359822


In [3]:
labeled_df = pd.read_csv("data/pre-trained-data.csv")
labeled_df = labeled_df.dropna(subset=['text', 'sentiment'])
df = df.dropna(subset=['text'])

labeled_data = [{"content": str(text), "label": int(label)} for text, label in zip(labeled_df["text"], labeled_df["sentiment"])]
unlabeled_data = [{"content": str(text)} for text in df["text"]]

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# 1. Model pretrained IndoBERT
MODEL_NAME = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)

# 2. Convert ke Dataset HuggingFace
labeled_ds = Dataset.from_list(labeled_data)
unlabeled_ds = Dataset.from_list(unlabeled_data)

# 3. Tokenization
def tokenize(batch):
    return tokenizer(batch["content"], truncation=True, padding="max_length", max_length=128)

labeled_ds = labeled_ds.map(tokenize, batched=True)
unlabeled_ds = unlabeled_ds.map(tokenize, batched=True)

# 4. Fine-tuning awal
training_args = TrainingArguments(
    output_dir="./train-checkpoint",
    per_device_train_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=labeled_ds,
    tokenizer=tokenizer,
)

trainer.train(resume_from_checkpoint="./train-checkpoint/checkpoint-189")
trainer.save_model("./indobert-finetuned")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 4155/4155 [00:00<00:00, 4734.38 examples/s]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16280\3046580120.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16280\3046580120.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


In [6]:
probs = trainer.predict(unlabeled_ds)

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
import torch
import torch.nn.functional as F

probs = F.softmax(torch.tensor(probs), dim=1) 
weights = torch.tensor([-1.0, 0.0, 1.0]) 
sentiment_scores = torch.matmul(probs, weights)

# labeling
pred_indices = torch.argmax(probs, dim=1)
label_map = {0: "negative", 1: "neutral", 2: "positive"}

df["sentiment_score"] = sentiment_scores.numpy()
df["sentiment_label"] = [label_map[i.item()] for i in pred_indices]
df.to_csv("data/tiktok_comments_with_sentiment.csv", index=False)

In [ ]:
df["sentiment_label"].value_counts()